# 05 Colab MD Simulation

In [ ]:
# access permission and check if gromacs works
%%bash
chmod +x "/content/drive/MyDrive/gmx/gromacs/bin/gmx"
source "/content/drive/MyDrive/gmx/gromacs/bin/GMXRC"
gmx -h

In [ ]:
#Installing py3Dmol using pip
!pip install py3Dmol

#Importing py3Dmol for safety
import py3Dmol

#Installing biopython using pip
!pip install biopython

In [ ]:
# Check present working directory
!pwd

In [ ]:
# Change Working Directory
import os
dir = "/content/drive/MyDrive/1aki/"
if not os.path.exists(dir):
    os.makedirs(dir)
%cd /content/drive/MyDrive/1aki/
!pwd

In [ ]:
# Download the mdp file first and make necessary changes in mdp parameters
%%bash
source "/content/drive/MyDrive/gmx/gromacs/bin/GMXRC"
wget -O ions.mdp https://raw.githubusercontent.com/paulshamrat/ColabMDA/main/mdps/ions.mdp
wget -O md.mdp https://raw.githubusercontent.com/paulshamrat/ColabMDA/main/mdps/md.mdp
wget -O minim.mdp https://raw.githubusercontent.com/paulshamrat/ColabMDA/main/mdps/minim.mdp
wget -O npt.mdp https://raw.githubusercontent.com/paulshamrat/ColabMDA/main/mdps/npt.mdp
wget -O nvt.mdp https://raw.githubusercontent.com/paulshamrat/ColabMDA/main/mdps/nvt.mdp

In [ ]:
#Importing your PDB file using biopython
import os
from Bio.PDB import *
pdbid = ['1aki']
pdbl = PDBList()
for s in pdbid:
  pdbl.retrieve_pdb_file(s, pdir='.', file_format ="pdb", overwrite=True)
  os.rename("pdb"+s+".ent", s+".pdb")

In [ ]:
# We will constantly need to source GMXRC for GROMACS to work
%%bash
source "/content/drive/MyDrive/gmx/gromacs/bin/GMXRC"


# This code snippet sources the GROMACS environment (GMXRC) to ensure the necessary GROMACS commands are available for the subsequent steps of the MD simulation.
grep -v HOH 1aki.pdb > 1AKI_clean.pdb

# This code snippet removes water molecules (HOH) from the PDB file, resulting in a cleaned version of the protein structure without the solvent molecules.
gmx pdb2gmx -f 1AKI_clean.pdb -o 1AKI_processed.gro -water spce -ignh -ff amber99sb-ildn

# This code snippet uses the pdb2gmx command to convert the cleaned PDB file into a processed GROMACS GRO file. The -water spce option specifies the water model, -ignh ignores the hydrogen atoms in the PDB file, and -ff amber99sb-ildn specifies the force field to be used.
gmx editconf -f 1AKI_processed.gro -o 1AKI_newbox.gro -c -d 1.0 -bt cubic

# This code snippet utilizes the editconf command to add a simulation box around the protein. The -c option centers the protein in the box, -d 1.0 specifies the distance between the protein and the box edges, and -bt cubic specifies a cubic box shape.
gmx solvate -cp 1AKI_newbox.gro -cs spc216.gro -o 1AKI_solv.gro -p topol.top

# This code snippet uses the solvate command to add water molecules to the simulation box around the protein. The -cp option specifies the input coordinate file (the box with the protein), -cs specifies the input solvent coordinate file (water), -o specifies the output solvated structure file, and -p specifies the output topology file.
gmx grompp -f ions.mdp -c 1AKI_solv.gro -p topol.top -o ions.tpr


# This code snippet prepares the system for the addition of ions by running the grompp command. The ions.mdp file, solvated structure (1AKI_solv.gro), and topology file (topol.top) are used as inputs to generate the binary input file (ions.tpr).
echo "SOL" > options
echo " " >> options

# This code snippet creates an options file to provide interactive options to the genion command in the subsequent step.
# Now we have an atomic-level description of our system in the binary file ions.tpr. We will pass this file to genion:
# When prompted, choose group 13 "SOL" for embedding ions.
gmx genion -s ions.tpr -o 1AKI_solv_ions.gro -p topol.top -pname NA -nname CL -neutral < options


# This code snippet utilizes the genion command to add ions (sodium and chloride) to the solvated system. The ions.tpr file, topology file (topol.top), and the options file are used as inputs. The -pname NA and -nname CL options specify the ion names as sodium (NA) and chloride (CL), respectively. The -neutral option ensures the system is neutralized.
# Assemble the binary input using grompp using [](this) input parameter file:
gmx grompp -f minim.mdp -c 1AKI_solv_ions.gro -p topol.top -o em.tpr

# This code snippet prepares the system for energy minimization using the grompp command. The minim.mdp file, solvated and ionized structure (1AKI_solv_ions.gro), and topology file (topol.top) are used as inputs to generate the binary input file (em.tpr).
#Using grompp to prepare our minimization MD
gmx grompp -f minim.mdp -c 1AKI_solv_ions.gro -p topol.top -o em.tpr

# This code snippet performs the same function as the previous snippet, preparing the system for energy minimization.
gmx mdrun -v -deffnm em -nb gpu -ntmpi 1


#This is a trick to provide interactive options to gmx
echo "Potential" > options
echo " " >> options

# Let's do a bit of analysis. The em.edr file contains all of the energy terms that GROMACS collects during EM.
# You can analyze any .edr file using the GROMACS energy module:
gmx energy -f em.edr -o potential.xvg < options
# At the prompt, type "10 0" to select Potential (10); zero (0) terminates input.


# We will call grompp and mdrun just as we did at the EM step:
gmx grompp -f nvt.mdp -c em.gro -r em.gro -p topol.top -o nvt.tpr
gmx mdrun -deffnm nvt -nb gpu -ntmpi 1


#This is a trick to provide interactive options to gmx
echo "Temperature" > options
echo " " >> options
# Let's analyze the temperature progression, again using energy:
gmx energy -f nvt.edr -o temperature.xvg < options
# Type "16 0" at the prompt to select the temperature of the system and exit.


# We will call grompp and mdrun just as we did for NVT equilibration.
gmx grompp -f npt.mdp -c nvt.gro -r nvt.gro -t nvt.cpt -p topol.top -o npt.tpr
gmx mdrun -deffnm npt -nb gpu -ntmpi 1



# This is a trick to provide interactive options to gmx
echo "Pressure" > options
echo " " >> options
# Let's analyze the pressure progression, again using energy:
gmx energy -f npt.edr -o pressure.xvg < options
# Type "18 0" at the prompt to select the pressure of the system and exit.



#This is a trick to provide interactive options to gmx
echo "Density" > options
echo " " >> options

# Let's take a look at density as well
gmx energy -f npt.edr -o density.xvg < options
# this time using energy and entering "24 0" at the prompt.




In [ ]:
# We will run a 1-ns MD simulation,
%%bash
source "/content/drive/MyDrive/gmx/gromacs/bin/GMXRC"
gmx grompp -f md.mdp -c npt.gro -t npt.cpt -p topol.top -o md_0_1.tpr

In [ ]:
# Assuming you have one GPU available, the mdrun command to make use of it is as simple as:
%%bash
source "/content/drive/MyDrive/gmx/gromacs/bin/GMXRC"
gmx mdrun -deffnm md_0_1 -nb gpu -ntmpi 1

In [ ]:
# append
%%bash
source "/content/drive/MyDrive/gmx/gromacs/bin/GMXRC"
gmx mdrun -v -cpi md_1.cpt -noappend -deffnm md_1 -gpu_id 0



In [ ]:
#Concatenation of xtc files
%%bash
source "/content/drive/MyDrive/gmx/gromacs/bin/GMXRC"
gmx trjcat -f md_1.xtc md_1.part0002.xtc md_1.part0003.xtc md_1.part0004.xtc md_1.part0005.xtc md_1.part0006.xtc md_1.part0007.xtc md_1.part0008.xtc md_1.part0009.xtc md_1.part0010.xtc md_1.part0011.xtc md_1.part0012.xtc md_1.part0013.xtc -o md_1_all.xtc
